In [19]:
#import open_clip
#from transformers import CLIPTokenizer, CLIPTextModel
#from safetensors import safe_open
#import matplotlib.pyplot as plt
#import matplotlib
import tensorflow as tf
import numpy as np
import torch

import math
import sys
import os

In [20]:
%matplotlib inline

In [21]:
FILENAME = "C:\\Users\\James\\stable-diffusion-webui\\models\\Stable-diffusion\\sd_xl_base_1.0.ckpt"
#FILENAME = "C:\\Users\\James\\stable-diffusion-webui\\models\\Stable-diffusion\\sd_xl_base_1.0.safetensors"
# FILENAME = "C:\\Users\\James\\Downloads\\v1-5-pruned.ckpt"
weights = torch.load(FILENAME)
#weights_f = safe_open(FILENAME, framework="pt", device="cpu")
weights = {"state_dict": weights}
#for key in weights_f.keys():
#    weights['state_dict'][key] = weights_f.get_tensor(key)

In [22]:
def quick_gelu(x):
  return x * tf.sigmoid(x * 1.702)

In [23]:
class CLIPAttention(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(CLIPAttention, self).__init__()
    self.embed_dim = 768
    self.num_heads = 12
    self.head_dim = self.embed_dim // self.num_heads
    self.scale = self.head_dim**-0.5
    self.k_proj = tf.keras.layers.Dense(
        self.embed_dim,
        activation=None,
        weights=[
            weights["state_dict"][tname + ".k_proj.weight"].T,
            weights["state_dict"][tname + ".k_proj.bias"],
        ],
    )
    self.v_proj = tf.keras.layers.Dense(
        self.embed_dim,
        activation=None,
        weights=[
            weights["state_dict"][tname + ".v_proj.weight"].T,
            weights["state_dict"][tname + ".v_proj.bias"],
        ],
    )
    self.q_proj = tf.keras.layers.Dense(
        self.embed_dim,
        activation=None,
        weights=[
            weights["state_dict"][tname + ".q_proj.weight"].T,
            weights["state_dict"][tname + ".q_proj.bias"],
        ],
    )
    self.out_proj = tf.keras.layers.Dense(
        self.embed_dim,
        activation=None,
        weights=[
            weights["state_dict"][tname + ".out_proj.weight"].T,
            weights["state_dict"][tname + ".out_proj.bias"],
        ],
    )

  def _shape(self, tensor, seq_len: int, bsz: int):
    a = tf.reshape(tensor, (bsz, seq_len, self.num_heads, self.head_dim))
    # bs , n_head , seq_len , head_dim
    return tf.keras.layers.Permute((2, 1, 3))(a)

  def call(self, inputs):
    [hidden_states, causal_attention_mask] = inputs
    bsz, tgt_len, embed_dim = hidden_states.shape
    query_states = self.q_proj(hidden_states) * self.scale
    key_states = self._shape(self.k_proj(hidden_states), tgt_len, -1)
    value_states = self._shape(self.v_proj(hidden_states), tgt_len, -1)

    proj_shape = (-1, tgt_len, self.head_dim)
    query_states = self._shape(query_states, tgt_len, -1)
    query_states = tf.reshape(query_states, proj_shape)
    key_states = tf.reshape(key_states, proj_shape)

    src_len = tgt_len
    value_states = tf.reshape(value_states, proj_shape)
    attn_weights = query_states @ tf.keras.layers.Permute(
        (2, 1))(key_states)

    attn_weights = tf.reshape(
        attn_weights, (-1, self.num_heads, tgt_len, src_len))
    attn_weights = attn_weights + causal_attention_mask
    attn_weights = tf.reshape(attn_weights, (-1, tgt_len, src_len))

    attn_weights = tf.nn.softmax(attn_weights)
    attn_output = attn_weights @ value_states

    attn_output = tf.reshape(
        attn_output, (-1, self.num_heads, tgt_len, self.head_dim)
    )
    attn_output = tf.keras.layers.Permute((2, 1, 3))(attn_output)
    attn_output = tf.reshape(attn_output, (-1, tgt_len, embed_dim))

    attn_output = self.out_proj(attn_output)
    return attn_output

In [24]:
class OpenCLIPAttention(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(OpenCLIPAttention, self).__init__()
    self.embed_dim = 1280
    self.num_heads = 12
    self.head_dim = self.embed_dim // self.num_heads
    self.scale = self.head_dim**-0.5

    in_proj_weight = weights["state_dict"][tname + ".in_proj_weight"]
    in_proj_bias = weights["state_dict"][tname + ".in_proj_bias"]

    q_weight, k_weight, v_weight = tf.split(
        in_proj_weight, [self.embed_dim, self.embed_dim, self.embed_dim]
    )
    q_bias, k_bias, v_bias = tf.split(
        in_proj_bias, [self.embed_dim, self.embed_dim, self.embed_dim]
    )

    self.k_proj = tf.keras.layers.Dense(
        self.embed_dim, activation=None, weights=[k_weight, k_bias]
    )
    self.v_proj = tf.keras.layers.Dense(
        self.embed_dim, activation=None, weights=[v_weight, q_bias]
    )
    self.q_proj = tf.keras.layers.Dense(
        self.embed_dim, activation=None, weights=[q_weight, q_bias]
    )
    self.out_proj = tf.keras.layers.Dense(
        self.embed_dim,
        activation=None,
        weights=[
            weights["state_dict"][tname + ".out_proj.weight"].T,
            weights["state_dict"][tname + ".out_proj.bias"],
        ],
    )

  def _shape(self, tensor, seq_len: int, bsz: int):
    a = tf.reshape(tensor, (bsz, seq_len, self.num_heads, self.head_dim))
    # bs , n_head , seq_len , head_dim
    return tf.keras.layers.Permute((2, 1, 3))(a)

  def call(self, inputs):
    [hidden_states, causal_attention_mask] = inputs
    bsz, tgt_len, embed_dim = hidden_states.shape
    query_states = self.q_proj(hidden_states) * self.scale
    key_states = self._shape(self.k_proj(hidden_states), tgt_len, -1)
    value_states = self._shape(self.v_proj(hidden_states), tgt_len, -1)

    proj_shape = (-1, tgt_len, self.head_dim)
    query_states = self._shape(query_states, tgt_len, -1)
    query_states = tf.reshape(query_states, proj_shape)
    key_states = tf.reshape(key_states, proj_shape)

    src_len = tgt_len
    value_states = tf.reshape(value_states, proj_shape)
    attn_weights = query_states @ tf.keras.layers.Permute(
        (2, 1))(key_states)

    attn_weights = tf.reshape(
        attn_weights, (-1, self.num_heads, tgt_len, src_len))
    attn_weights = attn_weights + causal_attention_mask
    attn_weights = tf.reshape(attn_weights, (-1, tgt_len, src_len))

    attn_weights = tf.nn.softmax(attn_weights)
    attn_output = attn_weights @ value_states

    attn_output = tf.reshape(
        attn_output, (-1, self.num_heads, tgt_len, self.head_dim)
    )
    attn_output = tf.keras.layers.Permute((2, 1, 3))(attn_output)
    attn_output = tf.reshape(attn_output, (-1, tgt_len, embed_dim))

    attn_output = self.out_proj(attn_output)
    return attn_output

In [25]:
class CLIPMLP(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(CLIPMLP, self).__init__()
    self.fc1 = tf.keras.layers.Dense(
        3072,
        weights=[
            weights["state_dict"][tname + ".fc1.weight"].T,
            weights["state_dict"][tname + ".fc1.bias"],
        ],
    )
    self.fc2 = tf.keras.layers.Dense(
        768,
        weights=[
            weights["state_dict"][tname + ".fc2.weight"].T,
            weights["state_dict"][tname + ".fc2.bias"],
        ],
    )

  def __call__(self, hidden_states):
    hidden_states = self.fc1(hidden_states)
    hidden_states = quick_gelu(hidden_states)
    hidden_states = self.fc2(hidden_states)
    return hidden_states

In [26]:
class OpenCLIPMLP(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(OpenCLIPMLP, self).__init__()
    self.c_fc = tf.keras.layers.Dense(
        3072,
        weights=[
            weights["state_dict"][tname + ".c_fc.weight"].T,
            weights["state_dict"][tname + ".c_fc.bias"],
        ],
    )
    self.c_proj = tf.keras.layers.Dense(
        768,
        weights=[
            weights["state_dict"][tname + ".c_proj.weight"].T,
            weights["state_dict"][tname + ".c_proj.bias"],
        ],
    )

  def __call__(self, hidden_states):
    hidden_states = self.c_fc(hidden_states)
    hidden_states = gelu(hidden_states)
    hidden_states = self.c_proj(hidden_states)
    return hidden_states

In [27]:
class CLIPEncoderLayer(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(CLIPEncoderLayer, self).__init__()
    self.self_attn = CLIPAttention(tname + ".self_attn")
    self.layer_norm1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".layer_norm1.weight"],
            weights["state_dict"][tname + ".layer_norm1.bias"],
        ],
    )
    self.mlp = CLIPMLP(tname + ".mlp")
    self.layer_norm2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".layer_norm2.weight"],
            weights["state_dict"][tname + ".layer_norm2.bias"],
        ],
    )

  def __call__(self, inputs):
    [hidden_states, causal_attention_mask] = inputs
    residual = hidden_states

    hidden_states = self.layer_norm1(hidden_states)
    hidden_states = self.self_attn([hidden_states, causal_attention_mask])
    hidden_states = residual + hidden_states

    residual = hidden_states
    hidden_states = self.layer_norm2(hidden_states)
    hidden_states = self.mlp(hidden_states)

    hidden_states = residual + hidden_states

    return hidden_states

In [28]:
class OpenCLIPEncoderLayer(tf.keras.layers.Layer):
  def __init__(self, tname):
    super(OpenCLIPEncoderLayer, self).__init__()
    self.self_attn = OpenCLIPAttention(tname + ".attn")
    self.mlp = OpenCLIPMLP(tname + ".mlp")

    # ln_1 or attn_ln
    self.layer_norm1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".ln_1.weight"],
            weights["state_dict"][tname + ".ln_1.bias"],
        ],
    )
    # ln_2 or mlp_ln
    self.layer_norm2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".ln_2.weight"],
            weights["state_dict"][tname + ".ln_2.bias"],
        ],
    )

  def __call__(self, inputs):
    [hidden_states, causal_attention_mask] = inputs
    residual = hidden_states

    hidden_states = self.layer_norm1(hidden_states)
    hidden_states = self.self_attn([hidden_states, causal_attention_mask])
    hidden_states = residual + hidden_states

    residual = hidden_states
    hidden_states = self.layer_norm2(hidden_states)
    hidden_states = self.mlp(hidden_states)

    hidden_states = residual + hidden_states

    return hidden_states

In [29]:
class CLIPEncoder(tf.keras.layers.Layer):
  def __init__(self, tname, openclip=False):
    super(CLIPEncoder, self).__init__()
    if openclip:
      self.layers = [
          OpenCLIPEncoderLayer(tname + ".resblocks.%d" % i) for i in range(12)
      ]
    else:
      self.layers = [
          CLIPEncoderLayer(tname + ".layers.%d" % i) for i in range(12)
      ]

  def __call__(self, inputs):
    [hidden_states, causal_attention_mask] = inputs
    for i, l in enumerate(self.layers):
      hidden_states = l([hidden_states, causal_attention_mask])
    return hidden_states

In [30]:
class CLIPTextEmbeddings(tf.keras.layers.Layer):
  def __init__(self, tname, openclip=False, n_words=77):
    super(CLIPTextEmbeddings, self).__init__()

    token_emb_W = weights["state_dict"][tname + ".token_embedding.weight"]
    self.token_embedding_layer = tf.keras.layers.Embedding(
        49408,
        1280 if openclip else 768,
        name="token_embedding",
        weights=[token_emb_W],
    )

    pos_emb_W = weights["state_dict"][
        tname
        + (".positional_embedding" if openclip else ".position_embedding.weight")
    ]
    self.position_embedding_layer = tf.keras.layers.Embedding(
        n_words,
        1280 if openclip else 768,
        name="position_embedding",
        weights=[pos_emb_W],
    )

  def __call__(self, inputs):
    [input_ids, position_ids] = inputs
    word_embeddings = self.token_embedding_layer(input_ids)
    position_embeddings = self.position_embedding_layer(position_ids)
    return word_embeddings + position_embeddings

In [31]:
class CLIPTextTransformer(tf.keras.models.Model):
  def __init__(self, tname, openclip=False, n_words=77):
    super(CLIPTextTransformer, self).__init__()
    # Contains both token and position embedding
    self.embeddings = CLIPTextEmbeddings(
        tname + ("" if openclip else ".embeddings"), openclip, n_words=n_words
    )

    # Corresponds to blocks
    self.encoder = CLIPEncoder(
        tname + (".transformer" if openclip else ".encoder"), openclip
    )

    self.final_layer_norm = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][
                tname
                + (".ln_final.weight" if openclip else ".final_layer_norm.weight")
            ],
            weights["state_dict"][
                tname +
                (".ln_final.bias" if openclip else ".final_layer_norm.bias")
            ],
        ],
    )

    if openclip:
      self.text_projection = tf.keras.layers.Dense(
          512,
          weights=[
              weights["state_dict"][tname + ".text_projection.weight"].T,
              weights["state_dict"][tname + ".text_projection.bias"],
          ],
      )
    else:
      self.text_projection = None

    self.causal_attention_mask = tf.constant(
        np.triu(np.ones((1, 1, 77, 77), dtype=np.float32) * -np.inf, k=1)
    )

  def __call__(self, inputs, training=False):
    [input_ids, inp_position_ids] = inputs
    x = self.embeddings([input_ids, inp_position_ids])
    x = self.encoder([x, self.causal_attention_mask])

    # TODO: figure out the format of text
    # if self.text_projection is not None:
    # eot_indices = tf.argmax(text, axis=1)
    # normed = self.final_layer_norm(x)
    # o = tf.squeeze(tf.gather(normed, eot_indices, axis=1), axis=1)

    # t_proj = self.text_projection.val()
    #    pooled = o #tf.matmul(o, t_proj)
    # else:
    #    pooled = o

    return self.final_layer_norm(x)

In [32]:
def get_conv2d(tname, in_channels, out_channels, kernel_size, padding=0, stride=1):
  W = weights["state_dict"][tname + ".weight"].numpy()
  if len(W.shape) > 2:
    W = np.transpose(W, (2, 3, 1, 0))
  else:
    # raise Exception("Sigh....")
    W = np.transpose(W)
  assert in_channels == W.shape[-2]
  assert out_channels == W.shape[-1]
  b = weights["state_dict"][tname + ".bias"]

  r = tf.keras.models.Sequential(
      [
          tf.keras.layers.ZeroPadding2D(
              padding=(padding, padding), data_format=None),
          tf.keras.layers.Conv2D(
              out_channels, kernel_size, strides=(
                  stride, stride), weights=[W, b]
          ),
      ]
  )
  return r

In [33]:
def apply_seq(x, layers):
  for l in layers:
    x = l(x)
  return x

In [34]:
# not to be confused with ResnetBlock
class ResBlock(tf.keras.layers.Layer):
  def __init__(self, tname, channels, emb_channels, out_channels):
    super(ResBlock, self).__init__()
    self.in_layers = [
        tf.keras.layers.GroupNormalization(
            epsilon=1e-5,
            weights=[
                weights["state_dict"][tname + ".in_layers.0.weight"],
                weights["state_dict"][tname + ".in_layers.0.bias"],
            ],
        ),
        tf.keras.activations.swish,
        get_conv2d(tname + ".in_layers.2", channels,
                   out_channels, 3, padding=1),
    ]
    self.emb_layers = [
        tf.keras.activations.swish,
        tf.keras.layers.Dense(
            out_channels,
            weights=[
                weights["state_dict"][tname + ".emb_layers.1.weight"].T,
                weights["state_dict"][tname + ".emb_layers.1.bias"],
            ],
        ),
    ]
    self.out_layers = [
        tf.keras.layers.GroupNormalization(
            epsilon=1e-5,
            weights=[
                weights["state_dict"][tname + ".out_layers.0.weight"],
                weights["state_dict"][tname + ".out_layers.0.bias"],
            ],
        ),
        tf.keras.activations.swish,
        lambda x: x,
        get_conv2d(
            tname + ".out_layers.3", out_channels, out_channels, 3, padding=1
        ),
    ]
    self.skip_connection = (
        get_conv2d(tname + ".skip_connection", channels, out_channels, 1)
        if channels != out_channels
        else lambda x: x
    )

  def __call__(self, inputs):
    x, emb = inputs
    h = apply_seq(x, self.in_layers)
    emb_out = apply_seq(emb, self.emb_layers)
    h = h + emb_out[:, None, None]
    h = apply_seq(h, self.out_layers)
    ret = self.skip_connection(x) + h
    return ret

In [35]:
def td_dot(a, b):
  assert len(a.shape) == 4
  assert len(a.shape) == 4
  assert a.shape[0] == b.shape[0]
  assert b.shape[1] == a.shape[1]
  aa = tf.reshape(a, (-1, a.shape[2], a.shape[3]))
  bb = tf.reshape(b, (-1, b.shape[2], b.shape[3]))
  cc = tf.keras.backend.batch_dot(aa, bb)
  c = tf.reshape(cc, (-1, a.shape[1], cc.shape[1], cc.shape[2]))
  return c

In [36]:
def gelu(self):
  return (
      0.5
      * self
      * (
          1
          + tf.keras.activations.tanh(
              self * 0.7978845608 * (1 + 0.044715 * self * self)
          )
      )
  )

In [37]:
class GEGLU(tf.keras.layers.Layer):
  def __init__(self, tname, dim_in, dim_out):
    super(GEGLU, self).__init__()

    self.proj = tf.keras.layers.Dense(
        dim_out * 2,
        weights=[
            weights["state_dict"][tname + ".proj.weight"].T,
            weights["state_dict"][tname + ".proj.bias"],
        ],
    )
    self.dim_out = dim_out

  def __call__(self, x):
    xp = self.proj(x)
    x, gate = xp[..., : self.dim_out], xp[..., self.dim_out:]
    ans = x * gelu(gate)

    return ans

In [38]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, tname, dim, mult=4):
    super(FeedForward, self).__init__()
    self.net = [
        GEGLU(tname + ".net.0", dim, dim * mult),
        lambda x: x,
        tf.keras.layers.Dense(
            dim,
            weights=[
                weights["state_dict"][tname + ".net.2.weight"].T,
                weights["state_dict"][tname + ".net.2.bias"],
            ],
        ),
    ]

  def __call__(self, x):
    return apply_seq(x, self.net)

In [ ]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, tname, query_dim, context_dim, n_heads, d_head):
    super(CrossAttention, self).__init__()
    self.to_q = tf.keras.layers.Dense( n_heads*d_head,use_bias=False ,weights=[weights['state_dict'][tname+".to_q.weight"].T  ])
    self.to_k = tf.keras.layers.Dense( n_heads*d_head,use_bias=False ,weights=[weights['state_dict'][tname+".to_k.weight"].T  ])
    self.to_v = tf.keras.layers.Dense( n_heads*d_head,use_bias=False ,weights=[weights['state_dict'][tname+".to_v.weight"].T  ])
    self.scale = d_head ** -0.5
    self.num_heads = n_heads
    self.head_size = d_head
    self.to_out = [tf.keras.layers.Dense( n_heads*d_head ,weights=[weights['state_dict'][tname+".to_out.0.weight"].T , weights['state_dict'][tname+".to_out.0.bias"] ])]

  def __call__(self, inputs):
    assert type(inputs) is list
    if len(inputs) == 1:
        inputs = inputs + [None]
    [x , context] = inputs
    context = x if context is None else context
    q,k,v = self.to_q(x), self.to_k(context), self.to_v(context)
    assert len(x.shape) == 3
    q = tf.reshape(q , ( -1 , x.shape[1], self.num_heads, self.head_size) )
    k = tf.reshape(k , ( -1 , context.shape[1] , self.num_heads, self.head_size) )
    v = tf.reshape(v, ( -1 , context.shape[1] , self.num_heads, self.head_size) )
    
    q = tf.keras.layers.Permute((2,1,3))(q)  # (bs, num_heads, time, head_size)
    k = tf.keras.layers.Permute((2,3,1))(k)  # (bs, num_heads, head_size, time)
    v = tf.keras.layers.Permute((2,1,3))(v)# (bs, num_heads, time, head_size)
    
    score = td_dot(q,k) * self.scale
    weights = tf.keras.activations.softmax(score)                   # (bs, num_heads, time, time)
    attention = td_dot(weights,v)  
    attention = tf.keras.layers.Permute((2,1,3))(attention) # (bs, time, num_heads, head_size)

    h_ = tf.reshape(attention, (-1 ,x.shape[1] , self.num_heads * self.head_size))
    return apply_seq(h_ , self.to_out)

In [60]:
class BasicTransformerBlock(tf.keras.layers.Layer):
  def __init__(self, tname, dim, context_dim, n_heads, d_head):
    super(BasicTransformerBlock, self).__init__()
    self.attn1 = CrossAttention(
        tname + ".attn1", dim, dim, n_heads, d_head)
    self.ff = FeedForward(tname + ".ff", dim)
    self.attn2 = CrossAttention(
        tname + ".attn2", dim, context_dim, n_heads, d_head)
    self.norm1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm1.weight"],
            weights["state_dict"][tname + ".norm1.bias"],
        ],
    )
    self.norm2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm2.weight"],
            weights["state_dict"][tname + ".norm2.bias"],
        ],
    )
    self.norm3 = tf.keras.layers.LayerNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm3.weight"],
            weights["state_dict"][tname + ".norm3.bias"],
        ],
    )

  def __call__(self, inputs):
    [x, context] = inputs

    x = self.attn1([self.norm1(x)]) + x
    x = self.attn2([self.norm2(x), context]) + x
    x = self.ff(self.norm3(x)) + x
    return x

In [56]:
class SpatialTransformer(tf.keras.layers.Layer):
  def __init__(
      self, tname, depth, use_linear, channels, context_dim, n_heads, d_head
  ):
    super(SpatialTransformer, self).__init__()
    self.norm = tf.keras.layers.GroupNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm.weight"],
            weights["state_dict"][tname + ".norm.bias"],
        ],
    )

    self.use_linear = use_linear
    assert channels == n_heads * d_head
    if use_linear:
      self.proj_in = tf.keras.layers.Dense(
          channels,
          weights=[
              weights["state_dict"][tname + ".proj_in.weight"].T,
              weights["state_dict"][tname + ".proj_in.bias"],
          ],
      )
    else:
      self.proj_in = get_conv2d(
          tname + ".proj_in", channels, n_heads * d_head, 1)

    print(depth)
    self.transformer_blocks = [
        BasicTransformerBlock(
            tname + ".transformer_blocks.0", channels, context_dim, n_heads, d_head
        )
        for d in range(depth)
    ]

    if use_linear:
      self.proj_out = tf.keras.layers.Dense(
          channels,
          weights=[
              weights["state_dict"][tname + ".proj_out.weight"].T,
              weights["state_dict"][tname + ".proj_out.bias"],
          ],
      )
    else:
      self.proj_out = get_conv2d(
          tname + ".proj_out", n_heads * d_head, channels, 1
      )

  def __call__(self, inputs):
    [x, context] = inputs
    b, h, w, c = x.shape
    x_in = x
    x = self.norm(x)
    if not self.use_linear:
      x = self.proj_in(x)
    x = tf.reshape(x, (-1, h * w, c))
    if self.use_linear:
      x = self.proj_in(x)
    for block in self.transformer_blocks:
      x = block([x, context])

    if self.use_linear:
      x = self.proj_out(x)
    x = tf.reshape(x, (-1, h, w, c))
    if not self.use_linear:
      x = self.proj_out(x)
    ret = x + x_in
    return ret

In [57]:
class Downsample(tf.keras.layers.Layer):
  def __init__(self, tname, channels):
    super(Downsample, self).__init__()
    self.op = get_conv2d(tname + ".op", channels,
                         channels, 3, stride=2, padding=1)

  def __call__(self, x):
    return self.op(x)

In [53]:
class Upsample(tf.keras.layers.Layer):
  def __init__(self, tname, channels):
    super(Upsample, self).__init__()
    self.conv = get_conv2d(tname + ".conv", channels,
                           channels, 3, padding=1)
    self.ups = tf.keras.layers.UpSampling2D(size=(2, 2))

  def __call__(self, x):
    x = self.ups(x)
    return self.conv(x)

In [43]:
class AttnBlock(tf.keras.layers.Layer):
  def __init__(self, tname, in_channels):
    super(AttnBlock, self).__init__()
    self.norm = tf.keras.layers.GroupNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm.weight"],
            weights["state_dict"][tname + ".norm.bias"],
        ],
    )
    self.q = get_conv2d(tname + ".q", in_channels, in_channels, 1)
    self.k = get_conv2d(tname + ".k", in_channels, in_channels, 1)
    self.v = get_conv2d(tname + ".v", in_channels, in_channels, 1)
    self.proj_out = get_conv2d(
        tname + ".proj_out", in_channels, in_channels, 1)

  # copied from AttnBlock in ldm repo
  def __call__(self, x):
    h_ = self.norm(x)
    q, k, v = self.q(h_), self.k(h_), self.v(h_)

    # compute attention
    b, h, w, c = q.shape
    q = tf.reshape(q, (-1, h * w, c))  # b,hw,c
    k = tf.keras.layers.Permute((3, 1, 2))(k)
    k = tf.reshape(k, (-1, c, h * w))  # b,c,hw
    w_ = q @ k
    w_ = w_ * (c ** (-0.5))
    w_ = tf.keras.activations.softmax(w_)

    # attend to values
    v = tf.keras.layers.Permute((3, 1, 2))(v)
    v = tf.reshape(v, (-1, c, h * w))
    w_ = tf.keras.layers.Permute((2, 1))(w_)
    h_ = v @ w_
    h_ = tf.keras.layers.Permute((2, 1))(h_)
    h_ = tf.reshape(h_, (-1, h, w, c))
    return x + self.proj_out(h_)

In [44]:
class ResnetBlock(tf.keras.layers.Layer):
  def __init__(self, tname, in_channels, out_channels=None):
    super(ResnetBlock, self).__init__()
    self.norm1 = tf.keras.layers.GroupNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm1.weight"],
            weights["state_dict"][tname + ".norm1.bias"],
        ],
    )
    self.conv1 = get_conv2d(
        tname + ".conv1", in_channels, out_channels, 3, padding=1
    )
    self.norm2 = tf.keras.layers.GroupNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm2.weight"],
            weights["state_dict"][tname + ".norm2.bias"],
        ],
    )
    self.conv2 = get_conv2d(
        tname + ".conv2", out_channels, out_channels, 3, padding=1
    )
    self.nin_shortcut = (
        get_conv2d(tname + ".nin_shortcut", in_channels, out_channels, 1)
        if in_channels != out_channels
        else lambda x: x
    )

  def __call__(self, x):
    h = self.conv1(tf.keras.activations.swish(self.norm1(x)))
    h = self.conv2(tf.keras.activations.swish(self.norm2(h)))
    return self.nin_shortcut(x) + h

In [45]:
class Mid(tf.keras.layers.Layer):
  def __init__(self, tname, block_in):
    super(Mid, self).__init__()
    self.block_1 = ResnetBlock(tname + ".block_1", block_in, block_in)
    self.attn_1 = AttnBlock(tname + ".attn_1", block_in)
    self.block_2 = ResnetBlock(tname + ".block_2", block_in, block_in)

  def __call__(self, x):
    return apply_seq(x, [self.block_1, self.attn_1, self.block_2])

In [46]:
class Decoder(tf.keras.models.Model):
  def __init__(self, tname):
    super(Decoder, self).__init__()
    sz = [(128, 256), (256, 512), (512, 512), (512, 512)]
    self.conv_in = get_conv2d(tname + ".conv_in", 4, 512, 3, padding=1)
    self.mid = Mid(tname + ".mid", 512)
    self.upp = tf.keras.layers.UpSampling2D(size=(2, 2))

    self.post_quant_conv = get_conv2d(
        "first_stage_model.post_quant_conv", 4, 4, 1)

    arr = []
    for i, s in enumerate(sz):
      arr.append(
          {
              "block": [
                  ResnetBlock(tname + ".up.%d.block.0" % i, s[1], s[0]),
                  ResnetBlock(tname + ".up.%d.block.1" % i, s[0], s[0]),
                  ResnetBlock(tname + ".up.%d.block.2" % i, s[0], s[0]),
              ]
          }
      )
      if i != 0:
        arr[-1]["upsample"] = {
            "conv": get_conv2d(
                tname + ".up.%d.upsample.conv" % i, s[0], s[0], 3, padding=1
            )
        }
    self.up = arr

    self.norm_out = tf.keras.layers.GroupNormalization(
        epsilon=1e-5,
        weights=[
            weights["state_dict"][tname + ".norm_out.weight"],
            weights["state_dict"][tname + ".norm_out.bias"],
        ],
    )
    self.conv_out = get_conv2d(tname + ".conv_out", 128, 3, 3, padding=1)

  def __call__(self, x, training=False):
    x = self.post_quant_conv(1 / 0.18215 * x)

    x = self.conv_in(x)
    x = self.mid(x)

    for l in self.up[::-1]:
      for b in l["block"]:
        x = b(x)
      if "upsample" in l:
        # https://pytorch.org/docs/stable/generated/torch.nn.functional.interpolate.html ?
        bs, c, py, px = x.shape
        x = self.upp(x)
        x = l["upsample"]["conv"](x)

    return self.conv_out(tf.keras.activations.swish(self.norm_out(x)))

In [47]:
class UNetModel(tf.keras.models.Model):
  def __init__(
      self,
      tname,
      in_channels,
      out_channels,
      model_channels,
      attention_resolutions,
      num_res_blocks,
      channel_mult,
      num_heads,
      num_head_channels,
      use_linear_in_transformer,
      transformer_depth,
      context_dim,
  ):
    super(UNetModel, self).__init__()
    time_embed_dim = model_channels * 4
    self.time_embed = [
        tf.keras.layers.Dense(
            time_embed_dim,
            weights=[
                weights["state_dict"][tname + ".time_embed.0.weight"].T,
                weights["state_dict"][tname + ".time_embed.0.bias"],
            ],
        ),
        tf.keras.activations.swish,
        tf.keras.layers.Dense(
            time_embed_dim,
            weights=[
                weights["state_dict"][tname + ".time_embed.2.weight"].T,
                weights["state_dict"][tname + ".time_embed.2.bias"],
            ],
        ),
    ]

    self.label_embed = [
        tf.keras.layers.Dense(
            time_embed_dim,
            weights=[
                weights["state_dict"][tname + ".label_emb.0.0.weight"].T,
                weights["state_dict"][tname + ".label_emb.0.0.bias"],
            ],
        ),
        tf.keras.activations.swish,
        tf.keras.layers.Dense(
            time_embed_dim,
            weights=[
                weights["state_dict"][tname + ".label_emb.0.2.weight"].T,
                weights["state_dict"][tname + ".label_emb.0.2.bias"],
            ],
        ),
    ]

    self.input_blocks = [
        [
            get_conv2d(
                "{}.input_blocks.0.0".format(tname),
                in_channels,
                model_channels,
                kernel_size=3,
                padding=1,
            )
        ]
    ]

    self.num_res_blocks = len(channel_mult) * [num_res_blocks]

    # print("input")
    input_block_chans = [model_channels]
    ch = model_channels
    ds = 1
    q = 1
    for level, mult in enumerate(channel_mult):
      for nr in range(self.num_res_blocks[level]):
        # print("resblock {}.input_blocks.{}.0".format(tname, q))
        layers = [
            [
                ResBlock(
                    "{}.input_blocks.{}.0".format(tname, q),
                    ch,
                    time_embed_dim,
                    mult * model_channels,
                )
            ]
        ]
        ch = mult * model_channels
        if ds in attention_resolutions:
          if num_head_channels == -1:
            dim_head = ch // num_heads
          else:
            num_heads = ch // num_head_channels
            dim_head = num_head_channels

          # print("spatial {}.input_blocks.{}.1".format(tname, q))
          # print(num_heads, dim_head)
          layers[-1].append(
              SpatialTransformer(
                  "{}.input_blocks.{}.1".format(tname, q),
                  transformer_depth[level],
                  use_linear_in_transformer,
                  ch,
                  context_dim,
                  num_heads,
                  dim_head,
              )
          )
        self.input_blocks.extend(layers)
        q += 1
        input_block_chans.append(ch)
      if level != len(channel_mult) - 1:
        out_ch = ch
        # print("downsample {}.input_blocks.{}.0".format(tname, q))
        self.input_blocks.append(
            [
                Downsample(
                    "{}.input_blocks.{}.0".format(tname, q), ch),
            ]
        )
        q += 1
        ch = out_ch
        input_block_chans.append(ch)
        ds *= 2

    if num_head_channels == -1:
      dim_head = ch // num_heads
    else:
      num_heads = ch // num_head_channels
      dim_head = num_head_channels

    self.middle_block = [
        ResBlock("{}.middle_block.0".format(
            tname), ch, time_embed_dim, ch),
        SpatialTransformer(
            "{}.middle_block.1".format(tname),
            transformer_depth[-1],
            use_linear_in_transformer,
            ch,
            context_dim,
            num_heads,
            dim_head,
        ),
        ResBlock("{}.middle_block.2".format(
            tname), ch, time_embed_dim, ch),
    ]

    q = 0
    # print("output")
    self.output_blocks = []
    for level, mult in list(enumerate(channel_mult))[::-1]:
      for i in range(self.num_res_blocks[level] + 1):
        blub = False
        ich = input_block_chans.pop()
        # print("resblock {}.output_blocks.{}.0".format(tname, q))
        layers = [
            [
                ResBlock(
                    "{}.output_blocks.{}.0".format(tname, q),
                    ch + ich,
                    time_embed_dim,
                    model_channels * mult,
                )
            ]
        ]
        ch = model_channels * mult
        if ds in attention_resolutions:
          if num_head_channels == -1:
            dim_head = ch // num_heads
          else:
            num_heads = ch // num_head_channels
            dim_head = num_head_channels

          blub = True
          # print("spatial {}.output_blocks.{}.1".format(tname, q))
          # print(num_heads, dim_head)
          layers[-1].append(
              SpatialTransformer(
                  "{}.output_blocks.{}.1".format(tname, q),
                  transformer_depth[level],
                  use_linear_in_transformer,
                  ch,
                  context_dim,
                  num_heads,
                  dim_head,
              )
          )
        if level and i == self.num_res_blocks[level]:
          out_ch = ch
          # print("upsample {}.output_blocks.{}.{}".format(tname, q, 2 if blub else 1))
          layers[-1].append(
              Upsample(
                  "{}.output_blocks.{}.{}".format(
                      tname, q, 2 if blub else 1),
                  ch,
              ),
          )
          ds //= 2
        q += 1
        self.output_blocks.extend(layers)

    # print(*self.input_blocks, sep='\n')
    # print()
    # print(*self.middle_block, sep='\n')
    # print()
    # print(*self.output_blocks, sep='\n')
    self.out = [
        tf.keras.layers.GroupNormalization(
            epsilon=1e-5,
            weights=[
                weights["state_dict"]["{}.out.0.weight".format(tname)],
                weights["state_dict"]["{}.out.0.bias".format(tname)],
            ],
        ),
        tf.keras.activations.swish,
        get_conv2d(
            "{}.out.2".format(tname),
            model_channels,
            out_channels,
            kernel_size=3,
            padding=1,
        ),
    ]

  def __call__(self, inputs, training=False):
    # TODO: real time embedding
    [x, t_emb, context, context_channel] = inputs
    t_emb_q = apply_seq(t_emb, self.time_embed)
    label_emb = apply_seq(context_channel, self.label_embed)
    emb = t_emb_q + label_emb

    def run(x, bb):
      print(bb)        
      if isinstance(bb, ResBlock):
        x = bb([x, emb])
      elif isinstance(bb, SpatialTransformer):
        x = bb([x, context])
      else:
        x = bb(x)
      return x

    print(emb)
    
    saved_inputs = []
    for i, b in enumerate(self.input_blocks):
      for bb in b:
        inppp = x
        x = run(x, bb)
      saved_inputs.append(x)

    print(self.input_blocks)
    print(self.middle_block)
    print(self.output_blocks)
      
    for bb in self.middle_block:
      x = run(x, bb)

    for i, b in enumerate(self.output_blocks):
      x = tf.concat([x, saved_inputs.pop()], axis=-1)
      for bb in b:
        x = run(x, bb)
    c = apply_seq(x, self.out)
    print(c)
    return c

In [48]:
text_max_len = 77


def get_text_enc():
  input_word_ids = tf.keras.layers.Input(
      shape=(text_max_len,), dtype=tf.int32)
  input_pos_ids = tf.keras.layers.Input(
      shape=(text_max_len,), dtype=tf.int32)
  embeds_1 = CLIPTextTransformer("conditioner.embedders.0.transformer.text_model")(
      [input_word_ids, input_pos_ids]
  )

  # TODO: Eventually migrate the Pytorch open_clip to here instead
  # embeds_2 = CLIPTextTransformer('conditioner.embedders.1.model', True)([input_word_ids, input_pos_ids])

  # return tf.keras.models.Model([input_word_ids,input_pos_ids], tf.concat([embeds_1, embeds_2], 2))
  return tf.keras.models.Model([input_word_ids, input_pos_ids], embeds_1)


def get_unet():
  t_emb = tf.keras.layers.Input((320,))
  latent = tf.keras.layers.Input((128, 128, 4))
  context = tf.keras.layers.Input((text_max_len, 2048))
  context_channel = tf.keras.layers.Input((2816,))

  # in_channels,
  # out_channels,
  # model_channels,
  # attention_resolutions,
  # num_res_blocks,
  # channel_mult,
  # num_heads
  # num_head_channels
  # transformer_depth
  # context_dim,
  unet = UNetModel(
      "model.diffusion_model",
      4,
      4,
      320,
      [4, 2],
      2,
      [1, 2, 4],
      -1,
      64,
      True,
      [1, 2, 10],
      2048,
  )
  # unet = UNetModel("model.diffusion_model", 4, 4, 320, [4, 2, 1], 2, [1, 2, 4, 4], 8, -1, 768)

  return tf.keras.models.Model(
      [latent, t_emb, context, context_channel], 
      unet([latent, t_emb, context, context_channel])
  )

def get_decoder():
  latent = tf.keras.layers.Input((128, 128, 4))
  decoder = Decoder("first_stage_model.decoder")
  return tf.keras.models.Model(latent, decoder(latent))

In [61]:
#text_encoder = get_text_enc()
unet = get_unet()
#decoder = get_decoder()

2
2
10
10
10
10
10
10
2
2
2
KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='tf.__operators__.add_14/AddV2:0', description="created by layer 'tf.__operators__.add_14'")


ValueError: You called `set_weights(weights)` on layer "multi_head_attention_210" with a weight list of length 3, but the layer was expecting 0 weights. Provided weights: [tensor([[ 0.0219,  0.0409, -0.0544,  ...,  0.0122...

In [ ]:
filtered = [x for x in weights["state_dict"].keys() if "label_embed" in x]
print(*filtered, sep="\n")

In [ ]:
# unet.save_weights("/tmp/unet.h5")
# decoder.save_weights("/tmp/decoder.h5")
# text_encoder.save_weights("/tmp/text_encoder.h5")

In [ ]:
prompt = "an astronaut riding a horse"

In [ ]:
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

In [ ]:
inputs = tokenizer([prompt], padding=True)

In [ ]:
model_openclip, _, _ = open_clip.create_model_and_transforms(
    "ViT-bigG-14", pretrained="laion2b_s39b_b160k"
)
tokenizer_openclip = open_clip.get_tokenizer("ViT-bigG-14")

In [ ]:
inputs_openclip = tokenizer_openclip([prompt]).detach().numpy()

In [ ]:
batch_size = 1

In [ ]:
# pos_ids = tf.convert_to_tensor(np.array(list(range(77)))[None].astype('int32'))
# pos_ids = np.repeat(pos_ids, batch_size, axis=0)

In [ ]:
phrase = inputs["input_ids"][0] + [49407] * (77 - len(inputs["input_ids"][0]))

phrase = np.array(phrase)[None].astype("int32")
phrase = np.repeat(phrase, batch_size, axis=0)
# phrase = tf.convert_to_tensor(phrase)
# context_clip = text_encoder([phrase])
# context_clip.shape

phrase = torch.from_numpy(phrase)
context_clip = model(phrase)
context_clip = context_clip.last_hidden_state.detach().numpy()
context_clip.shape

In [ ]:
phrase = [49406] * 77
phrase = np.array(phrase)[None].astype("int32")
phrase = np.repeat(phrase, batch_size, axis=0)

# phrase = tf.convert_to_tensor(phrase)
# unconditional_context_clip = text_encoder([phrase])
# unconditional_context_clip.shape

phrase = torch.from_numpy(phrase)
unconditional_context_clip = model(phrase)
unconditional_context_clip = (
    unconditional_context_clip.last_hidden_state.detach().numpy()
)
unconditional_context_clip.shape

In [ ]:
phrase = torch.from_numpy(inputs_openclip)
x = model_openclip.token_embedding(phrase)
x = x + model_openclip.positional_embedding
x = x.permute(1, 0, 2)  # NLD -> LND

outputs = {}
for i, r in enumerate(model_openclip.transformer.resblocks):
  if i == len(model_openclip.transformer.resblocks) - 1:
    outputs["penultimate"] = x.permute(1, 0, 2)  # LND -> NLD
  x = r(x, attn_mask=None)
outputs["last"] = x.permute(1, 0, 2)  # LND -> NLD
x = outputs

# x is a dict and will stay a dict
o = x["last"]
o = model_openclip.ln_final(o)
pooled = (
    o[torch.arange(o.shape[0]), phrase.argmax(dim=-1)
      ] @ model_openclip.text_projection
)
x["pooled"] = pooled

context_openclip = x["penultimate"].detach().numpy()
context_openclip_pooled = x["pooled"].detach().numpy()

context_openclip.shape

In [ ]:
aaa = tokenizer_openclip([""]).detach().numpy()
phrase = torch.from_numpy(aaa)
x = model_openclip.token_embedding(phrase)
x = x + model_openclip.positional_embedding
x = x.permute(1, 0, 2)  # NLD -> LND

outputs = {}
for i, r in enumerate(model_openclip.transformer.resblocks):
  if i == len(model_openclip.transformer.resblocks) - 1:
    outputs["penultimate"] = x.permute(1, 0, 2)  # LND -> NLD
  x = r(x, attn_mask=None)
outputs["last"] = x.permute(1, 0, 2)  # LND -> NLD
x = outputs

# x is a dict and will stay a dict
o = x["last"]
o = model_openclip.ln_final(o)
pooled = (
    o[torch.arange(o.shape[0]), phrase.argmax(dim=-1)
      ] @ model_openclip.text_projection
)
x["pooled"] = pooled

unconditional_context_openclip = x["penultimate"].detach().numpy()
unconditional_context_openclip_pooled = x["pooled"].detach().numpy()

unconditional_context_openclip.shape

In [ ]:
def timestep_embedding(timesteps, dim=320, max_period=10000):
  half = dim // 2
  freqs = np.exp(-math.log(max_period) *
                 np.arange(0, half, dtype=np.float32) / half)
  print(freqs.shape)
  args = np.array(timesteps) * freqs
  embedding = np.concatenate([np.cos(args), np.sin(args)])
  return (embedding).reshape(1, -1)

In [ ]:
context_full = tf.concat([context_clip, context_openclip], 2)
unconditional_context_full = tf.concat(
    [unconditional_context_clip, unconditional_context_openclip], 2
)

def conditioning_embedding(pooled_text_enc, dim, size, crop, ar):
    cat = tf.concat([size, crop, ar], axis=1)
    n_batch, w = cat.shape
    
    # Reshape cat to [n_batch * w]
    cat_reshaped = tf.reshape(cat, [n_batch * w])
    
    # Apply timestep_embedding and reshape the result
    embed = timestep_embedding(cat_reshaped, dim, 10000)
    embed_reshaped = tf.reshape(embed, [n_batch, w * dim])
    
    # Concatenate pooled_text_enc and embed along the second axis
    return tf.concat([pooled_text_enc, embed_reshaped], axis=1)

channel_context = conditioning_embedding(
  context_openclip_pooled,
  256,
  [1024,1024],
  [0,0],
  [1024,1024],
)
unconditional_context_channel = conditioning_embedding(
  unconditional_context_openclip_pooled,
  256,
  [1024,1024],
  [0,0],
  [1024,1024],
)

In [ ]:
def get_model_output(latent, t):
  # put into diffuser
  timesteps = np.array([t])
  t_emb = tf.convert_to_tensor(timestep_embedding(timesteps))
  t_emb = np.repeat(t_emb, batch_size, axis=0)
    
  conditional_latent = unet([latent, t_emb, context, context_channel]) 
  unconditional_latent = unet([latent, t_emb, unconditional_context, unconditional_context_channel])
  unconditional_guidance_scale = 7.5
  e_t = unconditional_latent + unconditional_guidance_scale * (
      conditional_latent - unconditional_latent
  )
  return e_t

In [ ]:
TIMESTEPS = int(os.getenv("TIMESTEPS", "25"))
timesteps = list(np.arange(1, 1000, 1000 // TIMESTEPS))
print(f"running for {timesteps} timesteps")

In [ ]:
linear_start = 0.00085
linear_end = 0.0120
num_timesteps = 1000
betas = tf.linspace(tf.sqrt(linear_start), tf.sqrt(
    linear_end), num_timesteps) ** 2
alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas, axis=0)

In [ ]:
print(alphas_cumprod.shape)

In [ ]:
alphas = [alphas_cumprod[t] for t in timesteps]
alphas_prev = [1.0] + alphas[:-1]

In [ ]:
def get_x_prev_and_pred_x0(x, e_t, index):
  temperature = 1
  a_t, a_prev = alphas[index], alphas_prev[index]
  sigma_t = 0
  sqrt_one_minus_at = math.sqrt(1 - a_t)
  # print(a_t, a_prev, sigma_t, sqrt_one_minus_at)

  pred_x0 = (x - sqrt_one_minus_at * e_t) / math.sqrt(a_t)

  # direction pointing to x_t
  dir_xt = math.sqrt(1.0 - a_prev - sigma_t**2) * e_t
  noise = sigma_t * tf.random.normal(x.shape) * temperature

  x_prev = math.sqrt(a_prev) * pred_x0 + dir_xt  # + noise
  return x_prev, pred_x0

In [ ]:
latent = tf.random.normal((batch_size, 128, 128, 4))

In [ ]:
for index, timestep in (t := tqdm(list(enumerate(timesteps))[::-1])):
  t.set_description("%3d %3d" % (index, timestep))
  e_t = get_model_output(latent, timestep)
  x_prev, pred_x0 = get_x_prev_and_pred_x0(latent, e_t, index)
  latent = x_prev

In [ ]:
decoded = decoder(latent)

In [ ]:
plt.imshow(np.clip((decoded[0].numpy() + 1) / 2, 0, 1))